## OCR dos adesivos via Deepseek

In [1]:
import os, io, sys, re
import torch

In [2]:
from tqdm import tqdm
from transformers import AutoModel, AutoTokenizer

In [3]:
import warnings
from transformers import logging
warnings.filterwarnings("ignore")
logging.set_verbosity_error()

In [4]:
input_dir = "validar-seabra"
output_file = "results.txt"

In [5]:
model_name = 'deepseek-ai/DeepSeek-OCR'

In [6]:
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
model = AutoModel.from_pretrained(model_name, trust_remote_code=True, use_safetensors=True)
model = model.eval().cuda().to(torch.bfloat16)

In [7]:
prompt = "<image>\n<|grounding|>OCR this image."

In [8]:
with open(output_file, "w", encoding="utf-8") as result:
    for file in tqdm(os.listdir(input_dir)):
    
        buffer = io.StringIO()
        sys_stdout = sys.stdout
        sys.stdout = buffer
    
        image_path = os.path.join(input_dir, file)
        _ = model.infer(tokenizer, prompt=prompt, image_file=image_path, output_path='results', crop_mode=True, save_results=False, test_compress=True)
    
        sys.stdout = sys_stdout
        
        output = buffer.getvalue()
        buffer.close()
    
        ref = re.search(r"<\|ref\|>(.*?)<\|/ref\|>", output)
        result.write(f"{file}:{ref.group(1).replace('.', '').replace('-', '').replace(':', '')}\n")